In [1]:
import json
import time

from application.aws import AwsAPI

# Latency test depending on context size

In this test, the model gets a list of messages which size increases on each iteration, and times to generate 1 sentence are measured (we measure 1 sentence as we have to pass the response to Polly and create an audio, and it works in the most efficient way if passed sentence by sentence). Here, Claude v2 was used to generate responses

In [2]:
aws = AwsAPI()

In [10]:
kills = []
with open("data/kills.csv") as f:
    for line in f:
        parts = line.split("\t")
        kill_str = parts[2].replace('""', '"')
        kill_str = kill_str[1:-1]
        kill_json = json.loads(kill_str)
        kills.append(kill_json)

for i in range(len(kills)):
    kills[i] = {
        "victim": {
            "username": f"random_user{i}",
        },
        "KillInstigator": {
            "username": "ProGamer69",
            "Distance": kills[i]["KillInstigator"]["Distance"],
            "first_kill": bool(kills[i]["KillInstigator"].get("first_kill", False)),
            "used_weapon": {
                "type": kills[i]["KillInstigator"]["used_weapon"]["type"],
                "name": kills[i]["KillInstigator"]["used_weapon"]["name"],
            },
            "Headshot": bool(kills[i]["KillInstigator"]["Headshot"]),
            "OneShot": bool(kills[i]["KillInstigator"]["OneShot"]),
        }
    }
print(kills[0])

{'victim': {'username': 'random_user0'}, 'KillInstigator': {'username': 'ProGamer69', 'Distance': 419.18713260946123, 'first_kill': True, 'used_weapon': {'type': 'Pistol', 'name': 'BerettaPistol'}, 'Headshot': False, 'OneShot': False}}


In [17]:
prompts = [
    "Imagine you're commenting the battle royale game match. You'll be getting kill events from the game like this one: "
    f"{json.dumps(kills[0])}. "
    "You'll need to comment on the kill event, using 3 senteces max. Try to also remember previous events and if you see some patterns feel free to voice them. Understood?",

    *(json.dumps(kill) for kill in kills)
]

In [ ]:
messages = []
for prompt in prompts[:100]:
    messages.append(
        {
            "role": "user",
            "content": [{"type": "text", "text": prompt}],
        }
    )
    response = aws.get_streamed_response_claude(messages, time.time())
    with open("dialogue.txt", "a") as file:
        file.write(f"User: {prompt}\nAssistant: {response}\n")
    messages.append(
        {
            "role": "assistant",
            "content": [{"type": "text", "text": response}],
        }
    )
    print(aws.get_bedrock_stats())

avg=0.60sec, min=0.59sec, max=0.60sec
avg=0.71sec, min=0.30sec, max=1.39sec
avg=0.78sec, min=0.52sec, max=1.24sec
avg=0.75sec, min=0.60sec, max=0.95sec
avg=0.87sec, min=0.30sec, max=1.33sec
avg=0.74sec, min=0.52sec, max=1.02sec
avg=0.93sec, min=0.60sec, max=1.54sec
avg=1.21sec, min=0.78sec, max=1.94sec
avg=0.81sec, min=0.52sec, max=1.57sec
avg=0.81sec, min=0.16sec, max=2.28sec
avg=0.78sec, min=0.35sec, max=1.67sec
avg=0.78sec, min=0.30sec, max=2.42sec
avg=0.76sec, min=0.30sec, max=2.06sec
avg=0.74sec, min=0.30sec, max=2.09sec
avg=0.71sec, min=0.23sec, max=2.33sec
avg=0.72sec, min=0.23sec, max=2.30sec
avg=0.81sec, min=0.23sec, max=2.61sec
avg=0.96sec, min=0.31sec, max=2.44sec
avg=0.88sec, min=0.46sec, max=2.85sec
avg=0.99sec, min=0.23sec, max=3.01sec
avg=1.14sec, min=0.31sec, max=3.49sec
avg=1.06sec, min=0.39sec, max=3.29sec
avg=0.96sec, min=0.15sec, max=3.52sec
avg=1.00sec, min=0.23sec, max=3.74sec
avg=0.94sec, min=0.23sec, max=3.69sec
avg=1.13sec, min=0.27sec, max=4.87sec
avg=1.08sec,

## Some ideas

things to add to init prompt:
- total number of players

things to add to kill event:
- num players alive (useful for battle royale)
- num kills by player
- previous victims usernames
- location on the map (in words, not coordinates)
- current top 5 players by kill count

for the silence moments:
 - we can send to unreal phrases where first 3 sentences are important and the rest is not so. 
 This way they can use the end of the phrase while waiting for the next one
 - we can also send events with game stats and ask to comment on them
